In [1]:
# Library Imports
import pandas as pd
from sqlalchemy import create_engine 
from Password import password

In [2]:
# Links to respective CSV Files
hospital_general_info = "Resources\Hospital_General_Information.csv"
patient_satisfaction = "Resources\cms_hospital_patient_satisfaction_2020.csv"

# Convert CSVs to DataFrames
hospital_general_info_df = pd.read_csv(hospital_general_info)
patient_satisfaction_df = pd.read_csv(patient_satisfaction)

C:\Users\cjrad\Anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3186: DtypeWarning: Columns (0,14,17,19,26) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Cleaning for Hospital General

In [3]:
# Confirm that Provider ID is unique value which is unrepeating throughout the dataset. 
# Suitable for Primary Key
hospital_general_info_df["Provider ID"].value_counts()

141311    1
453304    1
230077    1
60096     1
230081    1
         ..
521313    1
171373    1
370030    1
30064     1
141313    1
Name: Provider ID, Length: 4818, dtype: int64

In [4]:
# The purpose of this DF is to provide the address and contact information for various hospitals. Unneed Columns are stripped.
# The columns listed below were used in the General Hospital DataFrame

hospital_general_info_df = hospital_general_info_df[[
    "Provider ID",
    "Hospital Name",
    "Address",
    "City",
    "State",
    "ZIP Code",
    "County Name",
    "Phone Number",
    "Hospital Ownership",
    "Hospital Type",
    "Emergency Services",
    "Hospital overall rating"
]]

In [5]:
# Correct Object Datatypes into Strings, or other appropriate datatype. 
hospital_general_info_df.astype({
    "Hospital Name":"string",
    "Address":"string",
    "City":"string",
    "State":"string",
    "County Name":"string",
    "Hospital Ownership":"string",
    "Hospital Type":"string"
}).head()

,Provider ID,Hospital Name,Address,City,State,ZIP Code,County Name,Phone Number,Hospital Ownership,Hospital Type,Emergency Services,Hospital overall rating
0,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,2565938310,Government - Hospital District or Authority,Acute Care Hospitals,True,3.0
1,10032,WEDOWEE HOSPITAL,209 NORTH MAIN STREET,WEDOWEE,AL,36278,RANDOLPH,2563572111,Government - Hospital District or Authority,Acute Care Hospitals,True,4.0
2,10131,CRESTWOOD MEDICAL CENTER,ONE HOSPITAL DR SE,HUNTSVILLE,AL,35801,MADISON,2568823100,Proprietary,Acute Care Hospitals,True,3.0
3,20001,PROVIDENCE ALASKA MEDICAL CENTER,BOX 196604,ANCHORAGE,AK,99508,ANCHORAGE,9072123675,Voluntary non-profit - Church,Acute Care Hospitals,True,3.0
4,20018,YUKON KUSKOKWIM DELTA REG HOSPITAL,PO BOX 287,BETHEL,AK,99559,BETHEL,9075436300,Tribal,Acute Care Hospitals,True,3.0


In [6]:
# Column Renaming 
new_hospital_general_info_df = hospital_general_info_df.rename(columns={
    "Provider ID":"provider_id",
    "Hospital Name":"hospital_name",
    "Address":"address",
    "ZIP Code":"zip_code",
    "City":"city",
    "State":"state",
    "County Name":"county_name",
    "Phone Number":"phone_number",
    "Hospital Ownership":"hospital_ownership",
    "Hospital Type":"hospital_type",
    "Emergency Services":"emergency_services",
    "Hospital overall rating":"hospital_overall_rating"
})

In [7]:
# .info() used to identify columns which have missing data. county_name, hospital_overall_rating both have null values. 
new_hospital_general_info_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4818 entries, 0 to 4817
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   provider_id              4818 non-null   int64  
 1   hospital_name            4818 non-null   object 
 2   address                  4818 non-null   object 
 3   city                     4818 non-null   object 
 4   state                    4818 non-null   object 
 5   zip_code                 4818 non-null   int64  
 6   county_name              4803 non-null   object 
 7   phone_number             4818 non-null   int64  
 8   hospital_ownership       4818 non-null   object 
 9   hospital_type            4818 non-null   object 
 10  emergency_services       4818 non-null   bool   
 11  hospital_overall_rating  3648 non-null   float64
dtypes: bool(1), float64(1), int64(3), object(7)
memory usage: 418.9+ KB


In [8]:
# Fill the null values with 'N/A'.

new_hospital_general_info_df["hospital_overall_rating"].fillna("N/A", inplace=True)
new_hospital_general_info_df["county_name"].fillna("N/A", inplace=True)

# Cleaning for Patient Satisfaction

In [9]:
# Select appropriate columns.

patient_satisfaction_df = patient_satisfaction_df[['Facility ID', 'Facility Name', 'Address', 'City', 'State', 'County Name',
                             'Number of Completed Surveys', 'Survey Response Rate Percent', 'Start Date',
                             'End Date', 'Year', 'Hospital Type', 'Hospital overall rating',
                             'Mortality national comparison', 'Safety of care national comparison',
                             'Readmission national comparison', 'Patient experience national comparison',
                             'Effectiveness of care national comparison', 'Timeliness of care national comparison']]

In [10]:
# Rename Columns

new_patient_satisfaction_df = patient_satisfaction_df.rename(columns={"Facility ID": "facility_id", "Facility Name": "facility_name", "Address": "address",
                               "City": "city", "State": "state_", "County Name": "county_name",
                               "Number of Completed Surveys": "num_of_surveys",
                               "Survey Response Rate Percent": "survey_response_rate", "Start Date": "start_date",
                               "End Date": "end_date", "Year": "year_", "Hospital Type": "hospital_type",
                               "Hospital overall rating": "hospital_rating",
                               "Mortality national comparison": "mortality_national_comparison",
                               "Safety of care national comparison": "safety_of_care_national_comparison",
                               "Readmission national comparison": "readmission_national_comparison",
                               "Patient experience national comparison": "patient_experience_national_comparison",
                               "Effectiveness of care national comparison": "effectiveness_of_care_national_comparison",
                               "Timeliness of care national comparison": "timeliness_of_care_national_comparison"})


# Upload to Database

In [11]:
protocol = 'postgresql'
username = 'postgres'
password = password
host = 'localhost'
port = 5432
database_name = 'Hospital_DB'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [12]:
# Upload DataFrame to SQL Database

new_hospital_general_info_df.to_sql(name="hospital_location", con=engine, if_exists="replace", index=False)
new_patient_satisfaction_df.to_sql(name='ratings', con=engine, if_exists='append', index=False)

In [13]:
# Query the Data

pd.read_sql_query('select * from hospital_location', con=engine).head()

,provider_id,hospital_name,address,city,state,zip_code,county_name,phone_number,hospital_ownership,hospital_type,emergency_services,hospital_overall_rating
0,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,2565938310,Government - Hospital District or Authority,Acute Care Hospitals,True,3.0
1,10032,WEDOWEE HOSPITAL,209 NORTH MAIN STREET,WEDOWEE,AL,36278,RANDOLPH,2563572111,Government - Hospital District or Authority,Acute Care Hospitals,True,4.0
2,10131,CRESTWOOD MEDICAL CENTER,ONE HOSPITAL DR SE,HUNTSVILLE,AL,35801,MADISON,2568823100,Proprietary,Acute Care Hospitals,True,3.0
3,20001,PROVIDENCE ALASKA MEDICAL CENTER,BOX 196604,ANCHORAGE,AK,99508,ANCHORAGE,9072123675,Voluntary non-profit - Church,Acute Care Hospitals,True,3.0
4,20018,YUKON KUSKOKWIM DELTA REG HOSPITAL,PO BOX 287,BETHEL,AK,99559,BETHEL,9075436300,Tribal,Acute Care Hospitals,True,3.0
